In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# Load the trained model, scaler and onehotencoder pickle file
model = load_model('model.h5')

# load the encoder and scaler pickle file
with open('label_encoders.pkl','rb') as file:
    label_encoders = pickle.load(file)

with open('onehot_encoder.pkl','rb') as file:
    onehot_encoder = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
# Taking input data

input_data = {
    'customerID': ['123456'],
    'gender': ['Male'],
    'SeniorCitizen': ['No'],
    'Partner': ['Yes'],
    'Dependents': ['No'],
    'tenure': ['12'],
    'PhoneService': ['Yes'],
    'MultipleLines': ['No'],
    'InternetService': ['DSL'],
    'OnlineSecurity': ['Yes'],
    'OnlineBackup': ['No'],
    'DeviceProtection': ['No'],
    'TechSupport': ['No'],
    'StreamingTV': ['No'],
    'StreamingMovies': ['No'],
    'Contract': ['Month-to-month'],
    'PaperlessBilling': ['Yes'],
    'PaymentMethod': ['Electronic check'],
    'MonthlyCharges': ['29.85'],
    'TotalCharges': ['298.50'],
    'Churn': ['No']
}

In [4]:
# One Hot Encoding

# Convert input dictionary to DataFrame
input_df = pd.DataFrame(input_data)

# Select categorical columns that need encoding (same as used in training)
categorical_columns = ['InternetService', 'Contract', 'PaymentMethod']

# Apply OneHotEncoding to categorical columns
encoded_features = onehot_encoder.transform(input_df[categorical_columns])

# Convert the encoded data to DataFrame with proper column names
encoded_df = pd.DataFrame(encoded_features, columns=onehot_encoder.get_feature_names_out())

# Drop the original categorical columns from input data
input_df = input_df.drop(columns=categorical_columns)

# Merge input data with encoded features
final_input_df = pd.concat([input_df, encoded_df], axis=1)

# Print the final transformed input data
print(final_input_df)


  customerID gender SeniorCitizen Partner Dependents tenure PhoneService  \
0     123456   Male            No     Yes         No     12          Yes   

  MultipleLines OnlineSecurity OnlineBackup  ... InternetService_DSL  \
0            No            Yes           No  ...                 1.0   

  InternetService_Fiber optic InternetService_No Contract_Month-to-month  \
0                         0.0                0.0                     1.0   

  Contract_One year Contract_Two year PaymentMethod_Bank transfer (automatic)  \
0               0.0               0.0                                     0.0   

  PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                   0.0                             1.0   

   PaymentMethod_Mailed check  
0                         0.0  

[1 rows x 28 columns]


In [5]:
# List of categorical features to be Label Encoded
label_encoded_columns = [
    'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 
    'StreamingMovies', 'PaperlessBilling', 'Churn'
]

# Apply Label Encoding using the saved encoders
for col in label_encoded_columns:
    if col in label_encoders:  # Ensure the column exists in the saved encoders
        final_input_df[col] = label_encoders[col].transform(final_input_df[col])

# Print transformed input data
print(final_input_df)

ValueError: invalid literal for int() with base 10: 'No'